In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from data import *
from plotting import *
from regression import *
from utils import *

In [60]:
results_dir = 'results/updated-schema/'
os.makedirs(results_dir, exist_ok=True)

In [61]:
# Load data
pcd_df = load_pcd_df()

In [62]:
pcd_df

,System,Domain,Task,Publication date,Accessibility notes,Model accessibility,Dataset accessibility,Code accessibility,Link,Organization,...,Archived links,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Organization categorization (from Organization),Possibly over 1e23 FLOP,Training cost trends 2,Training cost trends 3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Theseus,Robotics,Maze solving,1950-07-02,NaN,NaN,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,Bell Laboratories,...,NaN,Industry,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN
2,SNARC,Robotics,Maze solving,1952-01-08,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,Harvard University,...,NaN,Academia,NaN,NaN,NaN,NaN,Academia,NaN,NaN,NaN
3,Genetic algorithm,Other,NaN,1954-07-02,NaN,NaN,NaN,NaN,https://link.springer.com/article/10.1007/BF01...,Institute for Advanced Study,...,NaN,Academia,NaN,NaN,NaN,NaN,Academia,NaN,NaN,NaN
4,Sequence-based pattern recognition,Vision,Character recognition,1955-03-01,NaN,NaN,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,Massachusetts Institute of Technology (MIT),...,NaN,Academia,NaN,NaN,NaN,NaN,Academia,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,Gemini 1.5 Flash,"Multimodal,Language,Vision,Audio","Chat,Audio speech recognition,Image captioning...",2024-05-10,NaN,API access,NaN,NaN,https://storage.googleapis.com/deepmind-media/...,Google DeepMind,...,NaN,Industry,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN
1544,Yi-Large,Language,"Chat,Language modelling/generation",2024-05-13,NaN,API access,NaN,NaN,NaN,01.AI,...,NaN,Industry,NaN,NaN,NaN,NaN,Industry,checked,NaN,NaN
1545,GPT-4o,"Multimodal,Language,Audio,Speech,Vision","Chat,Image generation,Audio generation,Vision-...",2024-05-13,NaN,API access,NaN,NaN,https://openai.com/index/hello-gpt-4o/,OpenAI,...,NaN,Industry,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN
1546,Chameleon-34B,"Multimodal,Image generation,Language,Vision","Language modelling/generation,Vision-language ...",2024-05-16,NaN,Unreleased,NaN,NaN,https://arxiv.org/abs/2405.09818v1,Facebook AI Research,...,NaN,Industry,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN


In [63]:
pcd_df.loc[pcd_df['System'] == 'Megatron-BERT']['Model accessibility']

648    Unreleased
Name: Model accessibility, dtype: object

In [64]:
access_df = pcd_df.dropna(subset=['Publication date', 'Training compute (FLOP)', 'Model accessibility'])
len(access_df)

504

In [65]:
access_df['Model accessibility'].unique()

array(['Unreleased', 'Hosted access (no API)',
       'Open access (non-commercial)', 'Open source', 'API access',
       'Open access (restricted use)'], dtype=object)

In [66]:
for cat in access_df['Model accessibility'].unique():
    print(cat, len(access_df.loc[access_df['Model accessibility'] == cat]))

Unreleased 186
Hosted access (no API) 10
Open access (non-commercial) 41
Open source 197
API access 26
Open access (restricted use) 44


In [67]:
open_access_categories = ['Open source', 'Open access (restricted use)', 'Open access (non-commercial)']
closed_access_categories = ['API access', 'Hosted access (no API)', 'Unreleased']

In [68]:
open_access_df = access_df.loc[access_df['Model accessibility'].isin(open_access_categories)]
len(open_access_df)

282

In [69]:
closed_access_df = access_df.loc[access_df['Model accessibility'].isin(closed_access_categories)]
len(closed_access_df)

222

In [70]:
# Add column with binary access label
access_df['Model open/closed'] = access_df['Model accessibility'].apply(
    lambda x: 'Open' if x in open_access_categories else 'Closed'
)

/var/folders/06/5htx0qw164j099bw2q7jk4gm0000gn/T/ipykernel_20101/148813227.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [71]:
access_df

,System,Domain,Task,Publication date,Accessibility notes,Model accessibility,Dataset accessibility,Code accessibility,Link,Organization,...,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Organization categorization (from Organization),Possibly over 1e23 FLOP,Training cost trends 2,Training cost trends 3,Model open/closed
97,Credibilty Network,Vision,"Character recognition,Image classification",1999-07-01,NaN,Unreleased,NaN,Unreleased,https://proceedings.neurips.cc/paper_files/pap...,"University College London (UCL),University of ...",...,"Academia,Academia",NaN,NaN,NaN,NaN,"Academia,Academia",NaN,NaN,NaN,Closed
209,LBL,Language,NaN,2012-06-27,NaN,Unreleased,NaN,Unreleased,https://arxiv.org/abs/1206.6426,University College London (UCL),...,Academia,NaN,NaN,NaN,NaN,Academia,NaN,NaN,NaN,Closed
232,RNN+weight noise+dynamic eval,Language,NaN,2013-08-04,NaN,Unreleased,NaN,Unreleased,https://arxiv.org/abs/1308.0850,University of Toronto,...,Academia,NaN,NaN,NaN,NaN,Academia,NaN,NaN,NaN,Closed
252,SPN-4+KN5,Language,NaN,2014-01-01,"code, no license specified: https://github.com...",Unreleased,NaN,Open access (non-commercial),https://spn.cs.washington.edu/papers/is14.pdf,"Singapore University of Technology & Design,DS...",...,"Academia,Government",NaN,NaN,NaN,NaN,"Academia,Government",NaN,NaN,NaN,Closed
272,Large regularized LSTM,Language,NaN,2014-09-08,Apache: https://github.com/wojzaremba/lstm,Unreleased,NaN,Open source,https://arxiv.org/abs/1409.2329,"New York University (NYU),Google Brain",...,"Academia,Industry",NaN,NaN,NaN,NaN,"Academia,Industry",NaN,NaN,NaN,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,phi-3-medium 14B,Language,"Chat,Language modelling/generation",2024-04-23,announced to be released later,Unreleased,NaN,NaN,https://arxiv.org/abs/2404.14219,Microsoft,...,Industry,NaN,NaN,NaN,NaN,Industry,checked,NaN,NaN,Closed
1542,Fugaku-LLM,Language,"Language modelling,Translation,Japanese langua...",2024-05-10,NaN,Open source,NaN,NaN,https://www.fujitsu.com/global/about/resources...,"Tohoku University,CyberAgent,Tokyo Institute o...",...,"Academia,Industry,Academia,Industry,Academia,I...",NaN,NaN,NaN,NaN,"Academia,Industry,Academia,Industry,Academia,I...",NaN,NaN,NaN,Open
1544,Yi-Large,Language,"Chat,Language modelling/generation",2024-05-13,NaN,API access,NaN,NaN,NaN,01.AI,...,Industry,NaN,NaN,NaN,NaN,Industry,checked,NaN,NaN,Closed
1546,Chameleon-34B,"Multimodal,Image generation,Language,Vision","Language modelling/generation,Vision-language ...",2024-05-16,NaN,Unreleased,NaN,NaN,https://arxiv.org/abs/2405.09818v1,Facebook AI Research,...,Industry,NaN,NaN,NaN,NaN,Industry,NaN,NaN,NaN,Closed


## Compute percentile filtering (skipped for now in regression/viz code)

In [72]:
outlier_window_size = 2  # years
start_large_scale_era = '2015-09-01'

In [73]:
pcd_df['Publication date'] = pd.to_datetime(pcd_df['Publication date'])
pcd_df.sort_values('Publication date', inplace=True)
pcd_df.dropna(subset=['Publication date', 'Notability criteria', 'Training compute (FLOP)'], inplace=True)

In [74]:
systems_by_percentile = {}
percentile_interval = 5
for percentile in range(95, -5, -percentile_interval):
  print(percentile)
  percentile_compute_low = np.zeros(len(pcd_df))
  percentile_compute_high = np.zeros(len(pcd_df))
  # Iterate through each row and calculate the 2-year moving average for each date
  for i, (index, row) in enumerate(pcd_df.iterrows()):
    # Define the 2-year window
    start_date = row['Publication date'] - pd.DateOffset(years=outlier_window_size/2)
    end_date = row['Publication date'] + pd.DateOffset(years=outlier_window_size/2)

    # Filter the DataFrame for this window
    window_df = pcd_df[(pcd_df['Publication date'] >= start_date) & (pcd_df['Publication date'] <= end_date)]

    percentile_compute_low[i] = np.percentile(window_df['Training compute (FLOP)'], percentile)
    percentile_compute_high[i] = np.percentile(window_df['Training compute (FLOP)'], percentile + percentile_interval)

  systems_flag = pcd_df['Training compute (FLOP)'] > np.array(percentile_compute_low)
  extra_systems_flag = pcd_df['Training compute (FLOP)'] <= np.array(percentile_compute_high)

  # raise Exception("Edit the following line if you want to consider models released after 2023-12-31.")
  extra_systems = pcd_df['System'][systems_flag & extra_systems_flag & (pcd_df['Publication date'] > pd.to_datetime('2015-09-30'))].values

  systems_by_percentile[percentile] = list(extra_systems)

95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10
5
0


In [75]:
systems_by_percentile

{95: ['GNMT',
  'AlphaGo Master',
  'AlphaGo Zero',
  'AlphaZero',
  'ResNeXt-101 32x48d',
  'Megatron-BERT',
  'OpenAI Five',
  'Meena',
  'GPT-3 175B (davinci)',
  'Megatron-Turing NLG 530B',
  'PaLM (540B)',
  'Minerva (540B)',
  'GPT-4',
  'PaLM 2',
  'Inflection-2',
  'Gemini 1.0 Ultra',
  'MegaScale (Production)'],
 90: ['NASv3 (CIFAR-10)',
  'FTW',
  'T5-11B',
  'AlphaStar',
  'mT5-XXL',
  'Switch',
  'Gopher (280B)',
  'ERNIE 3.0 Titan',
  'Chinchilla',
  'U-PaLM (540B)',
  'Flan-PaLM 540B',
  'GPT-3.5 (text-davinci-003)',
  'Claude 2',
  'Falcon-180B',
  'Inflection-2.5'],
 85: ['AlphaGo Fan',
  'AlphaGo Lee',
  'JFT',
  'AmoebaNet-A (F=448)',
  'XLNet',
  'Megatron-LM (8.3B)',
  'OpenAI Five Rerun',
  'Turing-NLG',
  'Yuan 1.0',
  'GLaM',
  'LaMDA',
  'OPT-175B',
  'Parti',
  'BLOOM-176B',
  'Grok-1',
  'Llama 3-70B'],
 80: ['OpenAI TI7 DOTA 1v1',
  'IMPALA',
  'BigGAN-deep 512x512',
  'GPT-2 (1.5B)',
  'iGPT-XL',
  'DALL-E',
  'Meta Pseudo Labels',
  'ProtT5-XXL',
  'ByT5-XX

In [76]:
compute_percentile_threshold = 50
selected_systems = []
for percentile in range(compute_percentile_threshold, 100, 5):
    selected_systems.extend(systems_by_percentile[percentile])
selected_systems

['Transformer',
 'GPT',
 'ProxylessNAS',
 'GBERT-Large',
 'wave2vec 2.0 LARGE',
 'AlphaFold 2',
 'DeBERTa',
 'HuBERT',
 'PLATO-XL',
 'Imagen',
 'ProGen2-xlarge',
 'NLLB',
 'ESM2-3B',
 'Flan-T5 11B',
 'LLaMA-7B',
 'LLaMA-13B',
 'WizardLM-7B',
 'Pangu-Weather',
 'DeepStack',
 'LSTM (Hebbian, Cache, MbPA)',
 'QT-Opt',
 'SciBERT',
 'DD-PPO',
 'ViT-Huge/14',
 'MSA Transformer',
 'M6-T',
 'ADM',
 'XGLM',
 'RETRO-7B',
 'Llama 2-7B',
 'FinGPT-13B',
 'Nemotron-3-8B',
 'ResNet-200',
 'Population-based DRL',
 'Mesh-TensorFlow Transformer 2.9B (translation)',
 'Megatron-LM (355M)',
 'T5-3B',
 'CamemBERT',
 'Noisy Student (L2)',
 'Once for All',
 'ERNIE 3.0',
 'BASIC-L',
 'Student of Games',
 'PaLI',
 'Whisper',
 'Taiyi-Stable Diffusion',
 'StarCoder',
 'WizardCoder-15.5B',
 'Llama 2-13B',
 'Skywork-13B',
 'MegaScale (175B)',
 'ResNet-152 (ImageNet)',
 'ConvS2S (ensemble of 8 models)',
 'PNASNet-5',
 'YOLOv3',
 'Transformer (Adaptive Input Embeddings) WT103',
 'BERT-Large-CAS (PTB+WT2+WT103)',
 'Co

# Top-n frontier filtering

In [77]:
access_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(access_df['Publication date'])
)

/var/folders/06/5htx0qw164j099bw2q7jk4gm0000gn/T/ipykernel_20101/3991044387.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
n = 10

access_df[f'top_{n}_at_release'] = False

for row, model in access_df.iterrows():
    # Filter for models released as of the model release, with the same open/closed status
    date_df = access_df[(access_df['Publication date (float)'] <= model['Publication date (float)']) 
        & (access_df['Model open/closed'] == model['Model open/closed'])]
    # Get the top n models (with the same open/closed status) by compute
    top_n_models = date_df.nlargest(n, 'Training compute (FLOP)')
    # Mark these models in the original dataframe
    access_df.loc[top_n_models.index, f'top_{n}_at_release'] = True

/var/folders/06/5htx0qw164j099bw2q7jk4gm0000gn/T/ipykernel_20101/1055558449.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [79]:
access_df[(access_df['top_10_at_release'] == True) & (access_df['Model open/closed'] == 'Open')]

,System,Domain,Task,Publication date,Accessibility notes,Model accessibility,Dataset accessibility,Code accessibility,Link,Organization,...,Training compute lower bound,Training compute upper bound,Training chip-hours,Organization categorization (from Organization),Possibly over 1e23 FLOP,Training cost trends 2,Training cost trends 3,Model open/closed,Publication date (float),top_10_at_release
366,SPIDER2,Biology,"Protein folding prediction,Proteins",2016-10-28,"some kind of download, unclear license\n\nhttp...",Open access (non-commercial),NaN,NaN,https://link.springer.com/protocol/10.1007/978...,"Griffith University,University of Iowa,Dezhou ...",...,NaN,NaN,NaN,"Academia,Academia,Academia",NaN,NaN,NaN,Open,2016.823924,True
372,BIDAF,Language,Question answering,2016-11-05,apache 2.0: https://github.com/allenai/bi-att-...,Open source,Open source,Open source,https://arxiv.org/abs/1611.01603v6,"University of Washington,Allen Institute for AI",...,NaN,NaN,480.0,"Academia,Research collective",NaN,NaN,NaN,Open,2016.844285,True
400,AlexNet + coordinating filters,Vision,Image classification,2017-03-28,license: https://github.com/wenwei202/caffe/bl...,Open source,NaN,Open source,https://arxiv.org/abs/1703.09746,"University of Pittsburgh,Duke University",...,NaN,NaN,NaN,"Academia,Academia",NaN,NaN,NaN,Open,2017.240590,True
472,ULM-FiT,Language,Text classification,2018-01-18,https://nlp.fast.ai/category/classification.html,Open source,NaN,Open source,https://arxiv.org/abs/1801.06146,"University of San Francisco,Insight Centre NUI...",...,NaN,NaN,NaN,"Academia,Academia",NaN,NaN,NaN,Open,2018.046544,True
497,DNCON2,Biology,"Proteins,Protein folding prediction",2018-05-01,license: https://github.com/multicom-toolbox/D...,Open source,Open source,Open source,https://academic.oup.com/bioinformatics/articl...,University of Missouri,...,NaN,NaN,NaN,Academia,NaN,NaN,NaN,Open,2018.333333,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,XVERSE-65B-2,Language,"Chat,Language modelling/generation",2023-12-08,Apache 2.0\n\nThe use of the source code in th...,Open access (restricted use),NaN,Open source,https://github.com/xverse-ai/XVERSE-65B/blob/m...,"XVERSE Technology,Shenzhen Yuanxiang Technology",...,NaN,NaN,NaN,"Industry,Industry",checked,NaN,NaN,Open,2023.935832,True
1477,DeepSeek LLM 67B,Language,Chat,2024-01-05,https://github.com/deepseek-ai/deepseek-LLM/bl...,Open access (restricted use),NaN,NaN,"https://arxiv.org/abs/2401.02954, https://gith...",DeepSeek,...,NaN,NaN,NaN,Industry,NaN,NaN,NaN,Open,2024.010952,True
1490,Code Llama-70B,Language,Code generation,2024-01-29,Llama 2 license. can't use outputs to train mo...,Open access (restricted use),NaN,NaN,https://ai.meta.com/research/publications/code...,Meta AI,...,NaN,NaN,2592000.0,Industry,NaN,NaN,NaN,Open,2024.076661,True
1518,DBRX,Language,"Chat,Code generation",2024-03-27,license: https://www.databricks.com/legal/open...,Open access (restricted use),Unreleased,Unreleased,https://www.databricks.com/blog/introducing-db...,Databricks,...,NaN,NaN,NaN,Industry,NaN,NaN,NaN,Open,2024.237852,True


In [80]:
access_df[(access_df['top_10_at_release'] == True) & (access_df['Model open/closed'] == 'Closed')]

,System,Domain,Task,Publication date,Accessibility notes,Model accessibility,Dataset accessibility,Code accessibility,Link,Organization,...,Training compute lower bound,Training compute upper bound,Training chip-hours,Organization categorization (from Organization),Possibly over 1e23 FLOP,Training cost trends 2,Training cost trends 3,Model open/closed,Publication date (float),top_10_at_release
97,Credibilty Network,Vision,"Character recognition,Image classification",1999-07-01,NaN,Unreleased,NaN,Unreleased,https://proceedings.neurips.cc/paper_files/pap...,"University College London (UCL),University of ...",...,NaN,NaN,NaN,"Academia,Academia",NaN,NaN,NaN,Closed,1999.500000,True
209,LBL,Language,NaN,2012-06-27,NaN,Unreleased,NaN,Unreleased,https://arxiv.org/abs/1206.6426,University College London (UCL),...,NaN,NaN,NaN,Academia,NaN,NaN,NaN,Closed,2012.487852,True
232,RNN+weight noise+dynamic eval,Language,NaN,2013-08-04,NaN,Unreleased,NaN,Unreleased,https://arxiv.org/abs/1308.0850,University of Toronto,...,NaN,NaN,NaN,Academia,NaN,NaN,NaN,Closed,2013.591547,True
252,SPN-4+KN5,Language,NaN,2014-01-01,"code, no license specified: https://github.com...",Unreleased,NaN,Open access (non-commercial),https://spn.cs.washington.edu/papers/is14.pdf,"Singapore University of Technology & Design,DS...",...,NaN,NaN,NaN,"Academia,Government",NaN,NaN,NaN,Closed,2014.000000,True
272,Large regularized LSTM,Language,NaN,2014-09-08,Apache: https://github.com/wojzaremba/lstm,Unreleased,NaN,Open source,https://arxiv.org/abs/1409.2329,"New York University (NYU),Google Brain",...,NaN,NaN,NaN,"Academia,Industry",NaN,NaN,NaN,Closed,2014.685832,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,Inflection-2,Language,Language modelling,2023-11-22,"via Pi, no API",Hosted access (no API),NaN,NaN,https://inflection.ai/inflection-2,Inflection AI,...,NaN,NaN,NaN,Industry,NaN,NaN,NaN,Closed,2023.890829,True
1453,Gemini 1.0 Ultra,"Multimodal,Language,Vision","Language modelling,Visual question answering,C...",2023-12-06,NaN,Hosted access (no API),NaN,NaN,https://storage.googleapis.com/deepmind-media/...,Google DeepMind,...,NaN,NaN,132000000.0,Industry,NaN,NaN,NaN,Closed,2023.930356,True
1502,MegaScale (Production),Language,Language modelling/generation,2024-02-23,Code for MegaScale (also called veScale) train...,Unreleased,NaN,NaN,https://arxiv.org/abs/2402.15627,"ByteDance,Peking University",...,NaN,NaN,NaN,"Industry,Academia",NaN,NaN,NaN,Closed,2024.143567,True
1504,Mistral Large,Language,Chat,2024-02-26,NaN,API access,NaN,NaN,https://mistral.ai/news/mistral-large/,Mistral AI,...,NaN,NaN,NaN,Industry,NaN,NaN,NaN,Closed,2024.151781,True


# Regression

In [112]:
# tentative
analysis_start_year = 2015
filtered_access_df = access_df[(access_df[f'top_{n}_at_release'] == True) & (access_df['Publication date (float)'] >= analysis_start_year)]


# filtered_access_df = access_df.loc[access_df['System'].isin(selected_systems)]

In [117]:
filtered_access_df.loc[filtered_access_df['Model open/closed'] == 'Closed',['System', 'Publication date (float)', 'Training compute (FLOP)']].to_csv('test.csv')

In [83]:
reg_results = fit_ols_regression(
    filtered_access_df,
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     214.5
Date:                Wed, 05 Jun 2024   Prob (F-statistic):           2.55e-25
Time:                        11:11:55   Log-Likelihood:                -70.996
No. Observations:                  90   AIC:                             146.0
Df Residuals:                      88   BIC:                             151.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1340.5167     93.122    -14.395      0.000   -1525.577   -1155.457
x1             0.6744      0.046     14.646      0.000       0.583       0.766
==============================================================================
Omnibus:                        4.938   Durbin-Watson:                   2.479
Prob(Omnibus):                  0.085   Jarque-Bera (JB):                4.995
Skew:                           0.555   Prob(JB):                       0.0823
Kurtosis:                       2.682   Cond. No.                     3.32e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.32e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [84]:
open_reg_results = fit_ols_regression(
    filtered_access_df[filtered_access_df['Model open/closed'] == 'Open'],
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
open_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     267.5
Date:                Wed, 05 Jun 2024   Prob (F-statistic):           2.94e-21
Time:                        11:11:55   Log-Likelihood:                -16.560
No. Observations:                  50   AIC:                             37.12
Df Residuals:                      48   BIC:                             40.94
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1292.5623     80.448    -16.067      0.000   -1454.314   -1130.811
x1             0.6506      0.040     16.354      0.000       0.571       0.731
==============================================================================
Omnibus:                        2.164   Durbin-Watson:                   2.344
Prob(Omnibus):                  0.339   Jarque-Bera (JB):                1.951
Skew:                           0.380   Prob(JB):                        0.377
Kurtosis:                       2.400   Cond. No.                     3.34e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.34e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [85]:
closed_reg_results = fit_ols_regression(
    filtered_access_df[filtered_access_df['Model open/closed'] == 'Closed'],
    ['Publication date (float)'],
    'Training compute (FLOP)',
    logy=True
)
closed_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     159.8
Date:                Wed, 05 Jun 2024   Prob (F-statistic):           3.47e-15
Time:                        11:11:55   Log-Likelihood:                -24.562
No. Observations:                  40   AIC:                             53.12
Df Residuals:                      38   BIC:                             56.50
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1470.2172    118.172    -12.441      0.000   -1709.445   -1230.990
x1             0.7388      0.058     12.642      0.000       0.620       0.857
==============================================================================
Omnibus:                        1.631   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.443   Jarque-Bera (JB):                1.485
Skew:                           0.447   Prob(JB):                        0.476
Kurtosis:                       2.699   Cond. No.                     3.29e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.29e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [86]:
print('All:')
print_growth_rates(reg_results)
print('Open:')
print_growth_rates(open_reg_results)
print('Closed:')
print_growth_rates(closed_reg_results)

All:
Adj. R^2=0.71
0.67 OOMs/year (95% CI: 0.58, 0.77)
4.7x/year (95% CI: 3.8x, 5.8x)
doubling time of 5 months (95% CI: 5, 6)
Open:
Adj. R^2=0.84
0.65 OOMs/year (95% CI: 0.57, 0.73)
4.5x/year (95% CI: 3.7x, 5.4x)
doubling time of 6 months (95% CI: 5, 6)
Closed:
Adj. R^2=0.80
0.74 OOMs/year (95% CI: 0.62, 0.86)
5.5x/year (95% CI: 4.2x, 7.2x)
doubling time of 5 months (95% CI: 4, 6)


In [87]:
closed_reg_results

## Predictions

In [88]:
pred_start_year = analysis_start_year
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'
num_preds = (pred_end_year - pred_start_year) * 12 + 1

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, num_preds)})
pred_years

,Publication date (float)
0,2020.000000
1,2020.083333
2,2020.166667
3,2020.250000
4,2020.333333
...,...
56,2024.666667
57,2024.750000
58,2024.833333
59,2024.916667


In [89]:
predicted_open_df = get_predictions(open_reg_results, pred_years, ['Publication date (float)'])
predicted_open_df['Publication date'] = predicted_open_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_open_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,21.591590,0.102931,21.384634,21.798546,20.869779,22.313400,2020.000000,2020-01-01
1,21.645804,0.100021,21.444698,21.846910,20.925649,22.365959,2020.083333,2020-01-31
2,21.700018,0.097138,21.504709,21.895327,20.981461,22.418576,2020.166667,2020-03-02
3,21.754233,0.094283,21.564664,21.943801,21.037214,22.471251,2020.250000,2020-04-01
4,21.808447,0.091459,21.624556,21.992338,21.092908,22.523985,2020.333333,2020-05-01
...,...,...,...,...,...,...,...,...
56,24.627589,0.106664,24.413126,24.842052,23.903590,25.351588,2024.666667,2024-09-01
57,24.681803,0.109625,24.461387,24.902219,23.956019,25.407588,2024.750000,2024-10-01
58,24.736018,0.112606,24.509609,24.962426,24.008391,25.463644,2024.833333,2024-10-31
59,24.790232,0.115604,24.557794,25.022670,24.060707,25.519757,2024.916667,2024-12-01


In [90]:
predicted_open_df.set_index('Publication date', inplace=True)

In [91]:
predicted_closed_df = get_predictions(closed_reg_results, pred_years, ['Publication date (float)'])
predicted_closed_df['Publication date'] = predicted_closed_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_closed_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,22.120182,0.144788,21.827075,22.413290,21.146332,23.094032,2020.000000,2020-01-01
1,22.181747,0.140594,21.897129,22.466366,21.210419,23.153076,2020.083333,2020-01-31
2,22.243313,0.136446,21.967093,22.519532,21.274412,23.212213,2020.166667,2020-03-02
3,22.304878,0.132346,22.036957,22.572799,21.338310,23.271445,2020.250000,2020-04-01
4,22.366443,0.128301,22.106712,22.626174,21.402113,23.330772,2020.333333,2020-05-01
...,...,...,...,...,...,...,...,...
56,25.567826,0.164287,25.235244,25.900409,24.581377,26.554276,2024.666667,2024-09-01
57,25.629391,0.168671,25.287936,25.970847,24.639915,26.618868,2024.750000,2024-10-01
58,25.690957,0.173080,25.340575,26.041338,24.698364,26.683549,2024.833333,2024-10-31
59,25.752522,0.177513,25.393165,26.111878,24.756726,26.748317,2024.916667,2024-12-01


In [92]:
predicted_closed_df.set_index('Publication date', inplace=True)

## Differences between trends

In [93]:
NUM_SAMPLES = 1000

In [94]:
open_reg_results.conf_int()

array([[-1.45431363e+03, -1.13081098e+03],
       [ 5.70586630e-01,  7.30555843e-01]])

In [95]:
median_date = datetime_to_float_year(pd.to_datetime(filtered_access_df['Publication date'])).median()
median_date

2022.2705343096363

In [96]:
compute_refs = {}
for key, value in {'open': open_reg_results, 'closed': closed_reg_results}.items():
    X = np.array([median_date, 1])  # dummy 1 value makes this work
    X = sm.add_constant(X)
    ref_preds = value.get_prediction(X).summary_frame()
    compute_ref_low = ref_preds['mean_ci_lower'][0]
    compute_ref_high = ref_preds['mean_ci_upper'][0]
    compute_refs[key] = compute_ref_low, compute_ref_high
compute_refs

{'open': (22.97093742302845, 23.166530952949376),
 'closed': (23.65001233279298, 23.945207001115296)}

In [97]:
# Open
open_ci = open_reg_results.conf_int()
open_slope = lognorm_from_90_ci(
    10**open_ci[1][0],
    10**open_ci[1][1],
    NUM_SAMPLES,
)
# Closed
closed_ci = closed_reg_results.conf_int()
closed_slope = lognorm_from_90_ci(
    10**closed_ci[1][0],
    10**closed_ci[1][1],
    NUM_SAMPLES,
)

In [98]:
print_median_and_ci(open_slope)
print_median_and_ci(closed_slope)

Median: 4.4 [90% CI: 3.7, 5.4]
Median: 5.4 [90% CI: 4.1, 7.2]


In [99]:
# Calculate when the closed compute trend was equal to the open compute trend today
current_date = '2024-05-01'
current_open_compute_df = predicted_open_df.loc[current_date]
current_open_compute = lognorm_from_90_ci(
    10**current_open_compute_df['mean_ci_lower'],
    10**current_open_compute_df['mean_ci_upper'],
    NUM_SAMPLES,
)
print_median_and_ci(current_open_compute)

Median: 2.6e+24 [90% CI: 1.6e+24, 4.1e+24]


In [100]:
# Same for closed
current_closed_compute_df = predicted_closed_df.loc[current_date]
current_closed_compute = lognorm_from_90_ci(
    10**current_closed_compute_df['mean_ci_lower'],
    10**current_closed_compute_df['mean_ci_upper'],
    NUM_SAMPLES,
)
print_median_and_ci(current_closed_compute)

Median: 2.1e+25 [90% CI: 1.1e+25, 4.1e+25]


In [101]:
compute_gap = np.log10(current_closed_compute) - np.log10(current_open_compute)
print(
    f'Open access models are {np.median(compute_gap):.1f} OOMs ' + 
    f'[{np.percentile(compute_gap, 2.5):.1f}; {np.percentile(compute_gap, 97.5):.1f}] ' + 
    'behind closed access models'
)

Open access models are 0.9 OOMs [0.5; 1.4] behind closed access models


In [102]:
# Calculate when the closed compute trend was equal to the open compute trend today

np.random.seed(42)
open_filtered_access_df =  filtered_access_df[filtered_access_df['Model open/closed'] == 'Open']
closed_filtered_access_df = filtered_access_df[filtered_access_df['Model open/closed'] == 'Closed']

n_bootstrap = 1000
open_filtered_access_bootstrap_indices = [np.arange(0, len(open_filtered_access_df))] + \
                                          [np.random.choice(np.arange(0, len(open_filtered_access_df)), size=len(open_filtered_access_df), replace=True) for _ in range(n_bootstrap)]
closed_filtered_access_bootstrap_indices = [np.arange(0, len(closed_filtered_access_df))] + \
                                            [np.random.choice(np.arange(0, len(closed_filtered_access_df)), size=len(closed_filtered_access_df), replace=True) for _ in range(n_bootstrap)]

current_date_float = 2024.35
results_date_float = []

for i in range(n_bootstrap+1):
    open_reg_results_current = fit_ols_regression(
        open_filtered_access_df.iloc[open_filtered_access_bootstrap_indices[i]],
        ['Publication date (float)'],
        'Training compute (FLOP)',
        logy=True
    )

    closed_reg_results_current = fit_ols_regression(
        closed_filtered_access_df.iloc[closed_filtered_access_bootstrap_indices[i]],
        ['Publication date (float)'],
        'Training compute (FLOP)',
        logy=True
    )

    alpha_o, beta_o = open_reg_results_current.params
    alpha_c, beta_c = closed_reg_results_current.params

    current_log10_compute = alpha_o + beta_o*current_date_float
    results_date_float.append((current_log10_compute - alpha_c)/beta_c)

In [103]:
print("Point estimate: %.4f" % (results_date_float[0]))
print("Standard error: %.4f years" % (np.std(results_date_float[1:])))

Point estimate: 2023.1151
Standard error: 0.1674 years


In [104]:
current_date = 2024.25
current_open_compute = open_reg_results.params[1] * current_date + open_reg_results.params[0]
current_open_compute

24.356517629775453

In [105]:
closed_compute_match_date = (current_open_compute - closed_reg_results.params[0]) / closed_reg_results.params[1]
closed_compute_match_date

2023.0270617050305

In [106]:
open_time_lag = current_date - closed_compute_match_date
print(f'Open access models are {open_time_lag:.1f} years behind closed access models')

Open access models are 1.2 years behind closed access models


# Plots

In [107]:
# filter out systems before start of analysis (2015)
access_df_recent = filtered_access_df[filtered_access_df['Publication date'] > pred_start_date]

fig = px.scatter(
    access_df_recent,
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model accessibility',
    hover_data=['System'],
    log_y=True,
)

save_plot(fig, results_dir, f'training_compute_by_model_accessibility {analysis_start_year}')

fig.show()

In [108]:
fig = px.scatter(
    # access_df,
    # access_df.loc[access_df_recent['System'].isin(selected_systems)],
    access_df_recent,
    x='Publication date',
    y='Training compute (FLOP)',
    color='Model open/closed',
    hover_data=['System'],
    log_y=True,
)

save_plot(fig, results_dir, f'training_compute_by_open_closed {analysis_start_year}')

fig.show()

In [109]:
predicted_open_df.index


DatetimeIndex(['2020-01-01', '2020-01-31', '2020-03-02', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-02', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-10-31', '2020-12-01',
               '2021-01-01', '2021-01-31', '2021-03-02', '2021-04-02',
               '2021-05-02', '2021-06-02', '2021-07-02', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-01-31', '2022-03-02', '2022-04-02',
               '2022-05-02', '2022-06-02', '2022-07-02', '2022-08-01',
               '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
               '2023-01-01', '2023-01-31', '2023-03-02', '2023-04-02',
               '2023-05-02', '2023-06-02', '2023-07-02', '2023-08-01',
               '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
               '2024-01-01', '2024-01-31', '2024-03-02', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-02', '2024-08-01',
      

In [110]:
fig = px.scatter(
    # access_df,
    # access_df.loc[access_df['System'].isin(selected_systems)],
    access_df_recent,
    x='Publication date (float)',
    y='Training compute (FLOP)',
    color='Model open/closed',
    hover_data=['System'],
    log_y=True,
)

# Marker color
# fig.update_traces(
#     marker=dict(
#         color='rgb(0,100,200)',
#     ),
#     selector=dict(mode='markers'),
# )

# Shade in CI
fig.add_scatter(
    x=predicted_open_df['Publication date (float)'],
    y=10**predicted_open_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_open_df['Publication date (float)'],
    y=10**predicted_open_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(230,100,0,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_open_df['Publication date (float)'],
    y=10**predicted_open_df['mean'],
    mode='lines',
    line=dict(color='rgb(230,100,0)'),
    name=f'Regression mean (growth rate: {10**open_reg_results.params[1]:.1f}x per year)',
)

fig.add_scatter(
    x=predicted_closed_df['Publication date (float)'],
    y=10**predicted_closed_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_closed_df['Publication date (float)'],
    y=10**predicted_closed_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_closed_df['Publication date (float)'],
    y=10**predicted_closed_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**closed_reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')

# title

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, f'open_closed_regression {analysis_start_year}')

fig.show()